In [2]:
# 未使用gps提取特征前的，使用模型有随机森林和lightGBM；
# 随机森林: 0.1216，lightGBM：0.1234
# 数据预处理，特征提取，简单地划分训练集为训练集和验证集并简单调参，训练，输出结果。
# 数据预处理包括训练集预处理和测试集预处理。
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%cd E:\python_project\MLPS7

e:\python_project\MLPS7


In [3]:
# 训练集数据处理0: 读取训练集
df1 = pd.read_csv('train_TTI.csv')
df1['time'] = pd.to_datetime(df1['time'])
#df1

,id_road,TTI,speed,time
0,276183,1.03472,69.6197,2019-01-01 00:00:00
1,276183,1.03506,69.6275,2019-01-01 00:10:00
2,276183,1.04166,69.1003,2019-01-01 00:20:00
3,276183,1.02642,70.1266,2019-01-01 00:30:00
4,276183,1.01442,70.9565,2019-01-01 00:40:00
...,...,...,...,...
296193,276738,1.44719,28.3267,2019-12-21 23:10:00
296194,276738,1.35854,30.1750,2019-12-21 23:20:00
296195,276738,1.37941,29.7186,2019-12-21 23:30:00
296196,276738,1.21284,33.7999,2019-12-21 23:40:00


In [4]:
# 训练集数据处理1: 处理时间缺失
df3 = pd.DataFrame()
period1 = pd.date_range('2019-01-01 00:00:00', '2019-03-31 23:50:00', freq='10T')
period2 = pd.date_range('2019-10-01 00:00:00', '2019-12-21 23:50:00', freq='10T')
period = period1.append(period2)
for name, group in df1.groupby('id_road'):
    group.set_index('time', inplace=True)
    group = group.reindex(period, method='nearest')
    group['time'] = group.index
    group.reset_index(drop=True, inplace=True)
    df3 = df3.append(group)
# df3.to_csv('train_TTI_fillna.csv', index=False)
#df3

,id_road,TTI,speed,time
0,275911,1.17708,61.1719,2019-01-01 00:00:00
1,275911,1.20540,59.7347,2019-01-01 00:10:00
2,275911,1.15301,62.4490,2019-01-01 00:20:00
3,275911,1.16649,61.7272,2019-01-01 00:30:00
4,275911,1.18073,60.9831,2019-01-01 00:40:00
...,...,...,...,...
24763,276738,1.44719,28.3267,2019-12-21 23:10:00
24764,276738,1.35854,30.1750,2019-12-21 23:20:00
24765,276738,1.37941,29.7186,2019-12-21 23:30:00
24766,276738,1.21284,33.7999,2019-12-21 23:40:00


In [5]:
# 训练集数据处理1: 6+6+3，6 TTI, 3 speed, 3 TTI(想要预测的值)
# 将来如果得到预测集的3 speed，则可以变为6+9+3
id_road = df3['id_road'].unique()   # 共12条road
index = pd.MultiIndex.from_product([id_road, period1.append(period2)])
columnsList=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5',
            'speed0','speed1','speed2','speed3','speed4','speed5', #'speed6','speed7', 'speed8',
            'TTI6','TTI7','TTI8']
df4 = pd.DataFrame(0, index=index, columns=columnsList)
for name, group in df3.groupby('id_road'):
    # df3行数：6*24*172*12, 解释：每天6*24条记录，172天，12条road
    matrix = np.zeros((6*24*172, 6+6+3))
    tti = group['TTI'].values
    speed = group['speed'].values
    i = 6
    for j in range(6, 6*24*90-2):   # 前90天
        matrix[i,0:6] = tti[j-6:j]
        matrix[i,6:12] = speed[j-6:j]
        matrix[i,12:15] = tti[j:j+3]
        i += 1
    i += 8
    for j in range(6*24*90+6, 6*24*172-2):  # 后172-90天
        matrix[i,0:6] = tti[j-6:j]
        matrix[i,6:12] = speed[j-6:j]
        matrix[i,12:15] = tti[j:j+3]
        i += 1
    df4.loc[name] = matrix
df4.index.names = ['id_road', 'time']
df4=df4.loc[df4['TTI0']>0]
#df4.to_csv('train_TTI_6+6+3.csv')
#df4

TTI0     TTI1     TTI2     TTI3     TTI4  \
id_road time                                                               
275911  2019-01-01 01:00:00  1.17708  1.20540  1.15301  1.16649  1.18073   
        2019-01-01 01:10:00  1.20540  1.15301  1.16649  1.18073  1.20200   
        2019-01-01 01:20:00  1.15301  1.16649  1.18073  1.20200  1.18198   
        2019-01-01 01:30:00  1.16649  1.18073  1.20200  1.18198  1.20954   
        2019-01-01 01:40:00  1.18073  1.20200  1.18198  1.20954  1.16853   
...                              ...      ...      ...      ...      ...   
276738  2019-12-21 22:50:00  1.74871  1.70574  1.52142  1.53454  1.64355   
        2019-12-21 23:00:00  1.70574  1.52142  1.53454  1.64355  1.65691   
        2019-12-21 23:10:00  1.52142  1.53454  1.64355  1.65691  1.41686   
        2019-12-21 23:20:00  1.53454  1.64355  1.65691  1.41686  1.44202   
        2019-12-21 23:30:00  1.64355  1.65691  1.41686  1.44202  1.44719   

                                TTI5   speed0   speed1   speed2   speed3  \
id_road time                                                               
275911  2019-01-01 01:00:00  1.20200  61.1719  59.7347  62.4490  61.7272   
        2019-01-01 01:10:00  1.18198  59.7347  62.4490  61.7272  60.9831   
        2019-01-01 01:20:00  1.20954  62.4490  61.7272  60.9831  59.9037   
        2019-01-01 01:30:00  1.16853  61.7272  60.9831  59.9037  60.9183   
        2019-01-01 01:40:00  1.17467  60.9831  59.9037  60.9183  59.5302   
...                              ...      ...      ...      ...      ...   
276738  2019-12-21 22:50:00  1.65691  23.4425  24.0330  26.9447  26.7143   
        2019-12-21 23:00:00  1.41686  24.0330  26.9447  26.7143  24.9423   
        2019-12-21 23:10:00  1.44202  26.9447  26.7143  24.9423  24.7413   
        2019-12-21 23:20:00  1.44719  26.7143  24.9423  24.7413  28.9330   
        2019-12-21 23:30:00  1.35854  24.9423  24.7413  28.9330  28.4283   

                              speed4   speed5     TTI6     TTI7     TTI8  
id_road time                                                              
275911  2019-01-01 01:00:00  60.9831  59.9037  1.18198  1.20954  1.16853  
        2019-01-01 01:10:00  59.9037  60.9183  1.20954  1.16853  1.17467  
        2019-01-01 01:20:00  60.9183  59.5302  1.16853  1.17467  1.13694  
        2019-01-01 01:30:00  59.5302  61.6195  1.17467  1.13694  1.13410  
        2019-01-01 01:40:00  61.6195  61.2974  1.13694  1.13410  1.09296  
...                              ...      ...      ...      ...      ...  
276738  2019-12-21 22:50:00  24.9423  24.7413  1.41686  1.44202  1.44719  
        2019-12-21 23:00:00  24.7413  28.9330  1.44202  1.44719  1.35854  
        2019-12-21 23:10:00  28.9330  28.4283  1.44719  1.35854  1.37941  
        2019-12-21 23:20:00  28.4283  28.3267  1.35854  1.37941  1.21284  
        2019-12-21 23:30:00  28.3267  30.1750  1.37941  1.21284  1.21629  

[297024 rows x 15 columns]

In [34]:
# 训练集数据处理2: 添加特征
import calendar
df5 = pd.read_csv('train_TTI_6+6+3.csv')
df5['time']=pd.to_datetime(df5['time'])
# 添加是否放假的特征
month_day=[[x.month,x.day] for x in df5['time']]
weekday=[calendar.weekday(x.year,x.month,x.day) for x in df5['time']]
holidayList=[[1,1],[2,4],[2,5],[2,6],[2,7],[2,8],[2,9],[2,10],[10,1],[10,2],[10,3],[10,4],[10,5],[10,6],[10,7]]
workdayList=[[2,2],[2,3],[10,12]]

isHoliday=np.zeros(np.size(weekday))
for i in range(np.size(weekday)):
    if month_day[i] in holidayList:
        isHoliday[i]=1.0
    elif weekday[i]>=5:
        isHoliday[i]=1.0
    else:
        isHoliday[i]=0.0
    if month_day[i] in workdayList:
        isHoliday[i]=0.0
df5['isHoliday']=isHoliday
df5['weekday']=weekday
df5.to_csv('train_TTI_6+6+3_more.csv',index=0)

In [27]:
# 弃用
# 训练集数据处理3: 分割训练集为小训练集和小验证集(小测试集)
# 将12月数据拿出来作为验证集
# 以上的cell可以不运行；
#df5 = pd.read_csv('train_TTI_6+6+3.csv')
df5 = pd.read_csv('train_TTI_6+6+3_more.csv')
df5['time']=pd.to_datetime(df5['time'])
df5['month']=[x.month for x in df5['time']]
train_train=df5.loc[df5['month']!=12]
train_train.drop(['month'],axis=1,inplace=True)
train_test=df5.loc[df5['month']==12]
train_test.drop(['month'],axis=1,inplace=True)
#train_train.to_csv('train_train_6+6+3.csv',index=0)
#train_test.to_csv('train_test_6+6+3.csv',index=0)

In [32]:
# 模型训练：随机森林
from sklearn.ensemble import RandomForestRegressor
train_train['hour']=[x.hour+x.minute/60 for x in train_train['time']] # 需要time列转化为datetime类
train_test['hour']=[x.hour+x.minute/60 for x in train_test['time']] # 需要time列转化为datetime类
# 去除8-22之外的数据    0.1391
#train_train=train_train.loc[train_train['hour']>=8.5]
#train_train=train_train.loc[train_train['hour']<=21] 
#train_test=train_test.loc[train_test['hour']>=8.5]
#train_test=train_test.loc[train_test['hour']<=21]
mae=0
for id_road,group in train_train.groupby('id_road'):
    #X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','hour','isHoliday']
    X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','hour','isHoliday','weekday']
    y_columns_list=['TTI6','TTI7','TTI8']
    X=group[X_columns_list].values
    y=group[y_columns_list].values
    model = RandomForestRegressor(max_features="auto",n_estimators=50, min_samples_leaf = 10)
    #model = DecisionTreeRegressor(criterion='mae',splitter='random',max_depth=10,min_samples_split=5,min_samples_leaf=4)
    #model = linear_model.LinearRegression()
    model.fit(X, y)
    date_test=train_test.loc[train_test['id_road']==id_road]
    X_test=date_test[X_columns_list].values
    y_test=date_test[y_columns_list].values
    y_predict=model.predict(X_test)
    err=np.abs(y_test-y_predict)
    mae+=np.sum(np.sum(err))
display(mae)
mae/=len(train_test)*3
mae

11477.455474708637

0.10549907598636514

In [55]:
# 模型训练
from lightgbm import LGBMRegressor
#train_train=train_train.loc[train_train['hour']>=8.5]
#train_train=train_train.loc[train_train['hour']<=21] 
train_test=train_test.loc[train_test['hour']>=8.5]
train_test=train_test.loc[train_test['hour']<=21]
train_train['hour']=[x.hour+x.minute/60 for x in train_train['time']] # 需要time列转化为datetime类
train_test['hour']=[x.hour+x.minute/60 for x in train_test['time']] # 需要time列转化为datetime类
mae=0
for id_road,group in train_train.groupby('id_road'):
    #X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','hour','isHoliday']
    X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','hour','isHoliday','weekday']
    y_columns_list=['TTI6','TTI7','TTI8']
    X=group[X_columns_list].values
    y=group[y_columns_list].values
    y0=y[:,0]
    y1=y[:,1]
    y2=y[:,2]
    model0= LGBMRegressor(num_leaves=31,max_depth=10,learning_rate=0.08,n_estimators=300)
    model1= LGBMRegressor(num_leaves=31,max_depth=10,learning_rate=0.08,n_estimators=300)
    model2= LGBMRegressor(num_leaves=31,max_depth=10,learning_rate=0.08,n_estimators=300)
    #model = RandomForestRegressor(max_features="auto",n_estimators=50, min_samples_leaf = 10)
    #model=DecisionTreeRegressor(criterion='mae',splitter='random',max_depth=10,min_samples_split=5,min_samples_leaf=4)
    #model = linear_model.LinearRegression()
    model0.fit(X, y0)
    model1.fit(X, y1)
    model2.fit(X, y2)
    date_test=train_test.loc[train_test['id_road']==id_road]
    X_test=date_test[X_columns_list].values
    y_test=date_test[y_columns_list].values
    #y_predict=model.predict(X_test)
    #err=np.abs(y_test-y_predict)
    #mae+=np.sum(np.sum(err))
    y0_predict=model0.predict(X_test)
    y1_predict=model1.predict(X_test)
    y2_predict=model2.predict(X_test)
    err0=np.abs(y_test[:,0]-y0_predict)
    err1=np.abs(y_test[:,1]-y1_predict)
    err2=np.abs(y_test[:,2]-y2_predict)
    mae+=np.sum(err0)+np.sum(err1)+np.sum(err2)
mae/=len(train_test)*3
mae

0.14367975148695783

In [41]:
# 测试集数据处理0:  将toPredict_nolabel.csv和toPredict_train_TTI.csv合并为一个直接可以给模型调用的文件
# 弃用
data_test = pd.read_csv('toPredict_Xy.csv')
data_test['time']=pd.to_datetime(data_test['time'])
data_test['time']=[x.hour+x.minute/60 for x in data_test['time']]
#display(data_test)

data_test=np.array(data_test)
m=np.size(data_test,0)/9
print("m需要是整数，m=",m)
m=int(m)
Xy_test=np.zeros([m,14+3])
for i in range(m):
    group=data_test[i*9+0:i*9+9]
    ttl=group[0:6,1]
    id_road=group[0,0]
    speed=group[0:6,2]
    id_sample=group[6:9,4]
    time=group[6,3]
    Xy_test[i]=[id_road,ttl[0],ttl[1],ttl[2],ttl[3],ttl[4],ttl[5],speed[0],speed[1],speed[2],speed[3],speed[4],speed[5],time,group[6,4],group[7,4],group[8,4]]
Xy_test=pd.DataFrame(Xy_test,columns=['id_road','TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','hour','id_sample0','id_sample1','id_sample2'])
Xy_test.to_csv('test_6+6+3_old.csv',index=0)

m需要是整数，m= 1008.0


In [66]:
# 测试集数据处理0:  将toPredict_nolabel.csv和toPredict_train_TTI.csv合并为一个直接可以给模型调用的文件
# 现在在使用
data_test = pd.read_csv('toPredict_Xy.csv')
data_test['time']=pd.to_datetime(data_test['time'])
data_test['index']=data_test.index
data_test['index']=[int(x/9) for x in data_test['index']]
Xy_test_new=pd.DataFrame(columns=['id_road','time','TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','id_sample0','id_sample1','id_sample2'])
for index,group in data_test.groupby('index'):
    ttl=group.iloc[0:6,1].values
    id_road=group.iloc[0,0]
    speed=group.iloc[0:6,2].values
    id_sample=group.iloc[6:9,4].values
    time=group.iloc[6,3]
    Xy_test_new.loc[len(Xy_test_new)]=[id_road,time,ttl[0],ttl[1],ttl[2],ttl[3],ttl[4],ttl[5],speed[0],speed[1],speed[2],speed[3],speed[4],speed[5],group.iloc[6,4],group.iloc[7,4],group.iloc[8,4]]
Xy_test_new.to_csv('test_6+6+3.csv',index=0)

,id_road,time,TTI0,TTI1,TTI2,TTI3,TTI4,TTI5,speed0,speed1,speed2,speed3,speed4,speed5,id_sample0,id_sample1,id_sample2


In [69]:
# 测试集数据处理1: 添加特征
import calendar
Xy_test_new = pd.read_csv('test_6+6+3.csv')
Xy_test_new['time']=pd.to_datetime(Xy_test_new['time'])
# 添加是否放假的特征
month_day=[[x.month,x.day] for x in Xy_test_new['time']]
weekday=[calendar.weekday(x.year,x.month,x.day) for x in Xy_test_new['time']]
holidayList=[[1,1],[2,4],[2,5],[2,6],[2,7],[2,8],[2,9],[2,10],[10,1],[10,2],[10,3],[10,4],[10,5],[10,6],[10,7]]
workdayList=[[2,2],[2,3],[10,12]]

isHoliday=np.zeros(np.size(weekday))
for i in range(np.size(weekday)):
    if month_day[i] in holidayList:
        isHoliday[i]=1.0
    elif weekday[i]>=5:
        isHoliday[i]=1.0
    else:
        isHoliday[i]=0.0
    if month_day[i] in workdayList:
        isHoliday[i]=0.0
Xy_test_new['isHoliday']=isHoliday
Xy_test_new['weekday']=weekday
Xy_test_new.to_csv('test_6+6+3_more.csv',index=0)

In [79]:
#data_train=['id_road','time','TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','isHoliday','weekday']
#Xy_test=['id_road','time','TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','id_sample0','id_sample1','id_sample2','isHoliday','weekday']
data_train = pd.read_csv('train_TTI_6+6+3_more.csv')
data_train['time']=pd.to_datetime(data_train['time'])
data_train['hour']=[x.hour+x.minute/60 for x in data_train['time']]    # 只保留小时和分
Xy_test=pd.read_csv('test_6+6+3_more.csv')
Xy_test['time']=pd.to_datetime(Xy_test['time'])
Xy_test['hour']=[x.hour+x.minute/60 for x in Xy_test['time']]    # 只保留小时和分
y_predict=np.zeros(Xy_test.shape[0]*3)
for id_road,group in Xy_test.groupby('id_road'):
    group_train=data_train.loc[data_train['id_road']==id_road]
    X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','hour','isHoliday','weekday']
    y_columns_list=['TTI6','TTI7','TTI8']
    X=group_train[X_columns_list].values
    y=group_train[y_columns_list].values
    model = RandomForestRegressor(max_features="auto",n_estimators=50, min_samples_leaf = 10)
    #model=DecisionTreeRegressor(criterion='mae',splitter='random',max_depth=10,min_samples_split=5,min_samples_leaf=4)
    #model = linear_model.LinearRegression()
    model.fit(X,y)
    id_sample_columns_list=['id_sample0','id_sample1','id_sample2']
    X_test=group[X_columns_list].values
    id_sample=group[id_sample_columns_list].values
    y_test=model.predict(X_test)
    for i in range(np.size(y_test,0)):
        for j in range(np.size(y_test,1)):
            y_predict[int(id_sample[i,j])]=y_test[i,j]
y_predict

array([1.08204822, 1.10464171, 1.13459097, ..., 1.5367307 , 1.5140879 ,
       1.53518937])

In [41]:
#data_train=['id_road','time','TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','isHoliday','weekday']
#Xy_test=['id_road','time','TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','id_sample0','id_sample1','id_sample2','isHoliday','weekday']
data_train = pd.read_csv('train_TTI_6+6+3_more.csv')
data_train['time']=pd.to_datetime(data_train['time'])
data_train['hour']=[x.hour+x.minute/60 for x in data_train['time']]    # 只保留小时和分
Xy_test=pd.read_csv('test_6+6+3_more.csv')
Xy_test['time']=pd.to_datetime(Xy_test['time'])
Xy_test['hour']=[x.hour+x.minute/60 for x in Xy_test['time']]    # 只保留小时和分
y_predict=np.zeros(Xy_test.shape[0]*3)
for id_road,group in Xy_test.groupby('id_road'):
    group_train=data_train.loc[data_train['id_road']==id_road]
    X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','hour','isHoliday','weekday']
    y_columns_list=['TTI6','TTI7','TTI8']
    X=group_train[X_columns_list].values
    y=group_train[y_columns_list].values
    y0=y[:,0]
    y1=y[:,1]
    y2=y[:,2]
    model0= LGBMRegressor(num_leaves=31,max_depth=10,learning_rate=0.08,n_estimators=100)
    model1= LGBMRegressor(num_leaves=31,max_depth=10,learning_rate=0.08,n_estimators=100)
    model2= LGBMRegressor(num_leaves=31,max_depth=10,learning_rate=0.08,n_estimators=100)
    model0.fit(X, y0)
    model1.fit(X, y1)
    model2.fit(X, y2)
    id_sample_columns_list=['id_sample0','id_sample1','id_sample2']
    X_test=group[X_columns_list].values
    id_sample=group[id_sample_columns_list].values
    y0_predict=model0.predict(X_test)
    y1_predict=model1.predict(X_test)
    y2_predict=model2.predict(X_test)
    
    for i in range(np.size(y0_predict)):
        y_predict[int(id_sample[i,0])]=y0_predict[i]
        y_predict[int(id_sample[i,1])]=y1_predict[i]
        y_predict[int(id_sample[i,2])]=y2_predict[i]
y_predict

array([1.07256574, 1.08833027, 1.09765947, ..., 1.60248519, 1.63985061,
       1.55906916])

In [44]:
# 输出y_predict
out=np.zeros([len(y_predict),2])
for i in range(len(y_predict)):
    out[i,0]=i
    out[i,1]=y_predict[i]
out=pd.DataFrame(out,columns=['id_sample','TTI'])
out.to_csv('submit8.csv',mode='w',index=False,header=True,encoding= 'utf-8')

In [19]:
# 检测某两次结果的mae
y1=pd.read_csv('submit16_5+14.csv')['TTI'].values
y2=pd.read_csv('submit19.csv')['TTI'].values
err=np.abs(y1-y2)
mae=np.sum(err)/len(err)
display(mae)

y1=pd.read_csv('submit19.csv')['TTI'].values
y2=pd.read_csv('submit14.csv')['TTI'].values
err=np.abs(y1-y2)
mae=np.sum(err)/len(err)
display(mae)
y1=pd.read_csv('submit5.csv')['TTI'].values
y2=pd.read_csv('submit14.csv')['TTI'].values
err=np.abs(y1-y2)
mae=np.sum(err)/len(err)
display(mae)
#0.03911232699352509
y1=pd.read_csv('submit19.csv')['TTI'].values
y2=pd.read_csv('submit5.csv')['TTI'].values
err=np.abs(y1-y2)
mae=np.sum(err)/len(err)
display(mae)
y3=(y1+y2)/2
err1=np.abs(y3-y1)
err2=np.abs(y3-y2)
mae1=np.sum(err1)/len(err1)
mae2=np.sum(err2)/len(err2)
display(mae1)
display(mae2)

y_predict=y3
# 输出y_predict
out=np.zeros([len(y_predict),2])
for i in range(len(y_predict)):
    out[i,0]=i
    out[i,1]=y_predict[i]
out=pd.DataFrame(out,columns=['id_sample','TTI'])
out.to_csv('submit20_5+19.csv',mode='w',index=False,header=True,encoding= 'utf-8')

0.021200328594284064

0.01992984153516499

0.03911232699352509

0.03353798162728882

0.01676899081364441

0.01676899081364441